In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
# 把dataset作为参数传给dataloader
# dataloader提供一层可迭代结构
# 定义batch size为64，dataloader每次迭代返回的是64条特征和对应的64个标签

for X, y in test_dataloader: #第一次调用dataset，每次拿64个
    #正常的停下条件是达到最大batch数
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break #这里只是检查一下是否正常工作

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module): #NeuralNetwork作为一个子类，继承nn.Module
    def __init__(self): #init定义网络的各个层
        super().__init__() #初始化，调用父类的构造函数
        self.flatten = nn.Flatten() #形状匹配上
        #Fashion-MINST输入形状是 (1,28,28)，转换成nn.Linear的28*28
        self.linear_relu_stack = nn.Sequential( #神经网络层
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x): #在forward函数定义数据如何向前传播
        x = self.flatten(x) #这里调用
        logits = self.linear_relu_stack(x) #未经过softmax的分类输出
        # 后面nn.CrossEntropyLoss自带LogSoftmax，提前softmax不好
        return logits

model = NeuralNetwork().to(device) # to(device)把模型参数整体迁移到MPS
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# 训练用损失函数衡量误差，反向传播算梯度，用优化器更新参数
loss_fn = nn.CrossEntropyLoss() #损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #优化器

In [9]:
def train(dataloader, model, loss_fn, optimizer): #训练循环
    size = len(dataloader.dataset)
    model.train() #训练前一定train，测试前一定eval
    for batch, (X, y) in enumerate(dataloader):
        # dataloader 返回：
        # batch：第几个batch
        # (X, y): 从dataloader返回的一个batch
        X, y = X.to(device), y.to(device) #保证模型和数据在同一个设备

        # Compute prediction error
        pred = model(X) #向前传播
        loss = loss_fn(pred, y) #计算loss

        # Backpropagation
        loss.backward() #反向传播
        optimizer.step() #修改参数的地方
        optimizer.zero_grad() #清理梯度，必须得清，torch的梯度是累加的

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # loss.item()把tensor转换成python数
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn): #测试
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() #测试必写
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs): #epoch嵌套循环
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Test Error: 
 Accuracy: 64.5%, Avg loss: 1.096146 

Done!


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
